In [2]:
import math
import metapy
import sys
import time
import pytoml
import numpy

In [3]:
idx = metapy.index.make_inverted_index('config.toml')
idx.num_docs()

30

In [4]:
idx.total_corpus_terms()

2568

In [5]:
idx.avg_doc_length()

85.5999984741211

In [6]:
fidx = metapy.index.make_forward_index('config.toml')
fidx.num_labels()

2

In [19]:
dset = metapy.classify.MulticlassDataset(fidx)
len(dset)

30

In [8]:
set([dset.label(instance) for instance in dset])

{'investment', 'movein'}

In [14]:
accumalator = 0
for i in range (1000):
    view = dset[0:len(dset)+1]
    view = metapy.classify.MulticlassDatasetView(dset)
    view.shuffle()
    #print("{} vs {}".format(view[0].id, dset[0].id))
    training = view[0:int(0.75*len(view))]
    testing = view[int(0.75*len(view)):len(view)+1]
    nb = metapy.classify.NaiveBayes(training)
    mtrx = nb.test(testing)
    #print(mtrx)
    #mtrx.print_stats()
    accumalator = accumalator + mtrx.accuracy()
naiveBayesAccurcy = accumalator/1000
print(" Naive Bayes (Avg Accuracy): "+ str(accumalator/1000))


 Naive Bayes (Avg Accuracy): 0.77025


In [45]:
accumalator = 0
for i in range (1000):
    view = dset[0:len(dset)+1]
    view = metapy.classify.MulticlassDatasetView(dset)
    view.shuffle()
    #print("{} vs {}".format(view[0].id, dset[0].id))
    training = view[0:int(0.75*len(view))]
    testing = view[int(0.75*len(view)):len(view)+1]
    ova = metapy.classify.OneVsAll(training, metapy.classify.SGD, loss_id='hinge')
    mtrx = ova.test(testing)
    #print(mtrx)
    #mtrx.print_stats()
    accumalator = accumalator + mtrx.accuracy()
SVMAccurcy = accumalator/1000
print(" SVM (Avg Accuracy): "+ str(accumalator/1000))

 SVM (Avg Accuracy): 0.819375


In [37]:
accumalator = 0
for i in range (1000):
    view = dset[0:len(dset)+1]
    view = metapy.classify.MulticlassDatasetView(dset)
    view.shuffle()
    #print("{} vs {}".format(view[0].id, dset[0].id))
    training = view[0:int(0.75*len(view))]
    testing = view[int(0.75*len(view)):len(view)+1]
    ova = metapy.classify.LogisticRegression(training)
    mtrx = ova.test(testing)
    #print(mtrx)
    #mtrx.print_stats()
    accumalator = accumalator + mtrx.accuracy()
LogisticRegressionAccurcy = accumalator/1000
print(" Logistic Regression (Avg Accuracy): "+ str(accumalator/1000))

 Logistic Regression (Avg Accuracy): 0.780875


In [50]:
query = metapy.index.Document()
num_results = 10
query = metapy.index.Document()
ranker = metapy.index.DirichletPrior(25)
ev = metapy.index.IREval('config.toml')
with open('config.toml', 'r') as fin:
    cfg_d = pytoml.load(fin)
    query_cfg = cfg_d['query-runner']
    query_start = query_cfg.get('query-id-start', 0)

    with open('investment-queries.txt') as query_file:
        for query_num, line in enumerate(query_file):
            query.content(line.strip())
            #print(query.content())
            results = ranker.score(idx, query, num_results)
            #print(results)
            #print(num_results)
            avg_p = ev.avg_p(results, query_start + query_num, num_results)
            #print("Query {} average precision: {}".format(query_num + 1, avg_p))
DPRetrieval = ev.map()
print(" Dirichlet Prior Retrieval: (MAP): "+ str(ev.map()))

 Dirichlet Prior Retrieval: (MAP): 0.8788888888888889


In [51]:
print(" Dirichlet Prior Retrieval: (MAP): "+ str(DPRetrieval))
print(" SVM (Avg Accuracy): "+ str(SVMAccurcy))
print(" Logistic Regression (Avg Accuracy): "+ str(LogisticRegressionAccurcy))
print(" Naive Bayes (Avg Accuracy): "+ str(naiveBayesAccurcy))

 Dirichlet Prior Retrieval: (MAP): 0.8788888888888889
 SVM (Avg Accuracy): 0.819375
 Logistic Regression (Avg Accuracy): 0.780875
 Naive Bayes (Avg Accuracy): 0.77025
